In [1]:
import numpy as np

In [2]:
# Collect the data
import csv
dir_name = "./pick-and-place/"
data = []

for k in range(20):
    results = []
    filename = dir_name+'/demo'+str(k+1)+'.txt'
    with open(filename, newline=None) as inputfile:
        for row in csv.reader(inputfile):
            results.append(row)
    data_ = []
    for i in range(len(results)):
        tmp = results[i][-5:-2]
        data_.append([float(el) for el in tmp])
    data.append(np.array(data_))
    

In [3]:
# Preprocess the data
data_prep = data[:]
idx = []
T = 70 # Time duration
for k in range(20):
    data_new = data[k]
    q = len(data_new)
    while q>T:
        diff_ = np.abs(data_new[1:,2]-data_new[:q-1,2])
        a = np.argmin(diff_) # row index of the minimum
        data_new = np.delete(data_new,a,0)
        q = len(data_new)
    data_prep[k] = np.copy(data_new)
    idx.append(len(data_prep[k]))

data = data_prep

In [4]:
# Find the index of the object location
# Idx1: Object1 location
# Idx2: Object2 location

Idx1 = []
Idx2 = []
for k in range(20):
    b = data[k]
    a = b[:,2]
    idx = 5
    cond = False
    while not cond:
        idx = idx+1
        cond1 = (a[idx]-a[idx-1])<=0 and (a[idx]-a[idx-2])<=0
        cond2 = (a[idx]-a[idx+1])<=0 and (a[idx]-a[idx+2])<=0
        cond = cond1 and cond2
        if idx+1>len(a):
            print("Unsuccessful in identifying the min")
            cond = True

    Idx1.append(idx)
    Idx2.append(len(b)-1)
    
print(Idx1)    
print(Idx2)


[30, 32, 27, 25, 29, 25, 28, 35, 28, 23, 35, 31, 32, 25, 30, 32, 23, 29, 26, 31]
[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69]


In [5]:
data[0].shape

(70, 3)

In [6]:
# Tensorial representation
p = 3 # Number of states
q = 3 # Number of TP-Coordinates
r = T
#Import the data into tensor structure
N = 20
tensor_data = [np.empty([r,p,q])]*N #
for k in range(N): # iterate over the 5 demonstrations
    data_ = data[k]
    obj1 = data_[Idx1[k],:] # first object coordinate
    obj2 = data_[Idx2[k],:] # second object coordinate
    tmp  = np.empty([r,p,q])
    for t in range(T):
        tmp[t,:,0]= data_[t,:]-obj1 
        tmp[t,:,1]= data_[t,:]-obj2 
        tmp[t,:,2]= data_[t,:]
        #tmp_ = tmp[:,[0,1,2],:]
    tensor_data[k] = tmp #scale the data by 100sualization b
    
        

In [7]:
tensor_data[0].shape

(70, 3, 3)

In [8]:
# Plotting tools
from matplotlib import interactive
interactive(True)
# for interractive plots in jupyter-notebook
%matplotlib widget
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [9]:
# Plot the trajectories
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax = fig.add_subplot(1, 2, 1, projection='3d')
k = int(np.random.randint(20,size=1))
X = data[k]
x = X[:,0]
y = X[:,1]
z = X[:,2]
ax.plot3D(x,y,z)
plt.show()

FigureCanvasNbAgg()

In [12]:
# Find the array-normal distribution
from tensor_tools import array3
t_tools = array3()
coef = 1
var = 1
pow_ = 1
I = t_tools.shape(tensor_data[0])
M, Cov, A = t_tools.anormal(tensor_data, coef, var, pow_, constraint=True, normalised=False, 
                for_mix_model=False,gamma_k=None)


Error EM:  239.9748695883926
Error EM:  147.04734387449264
Error EM:  188.29363412613236
Error EM:  189.51253029130137
Error EM:  162.42289840870208
Error EM:  125.66833181178804
Error EM:  91.4353246385369
Error EM:  64.16700135108348
Error EM:  44.079204826649026
Error EM:  29.8905854878145
Error EM:  20.10594469994628
Error EM:  13.454392609423751
Error EM:  8.972959225659674
Error EM:  5.970928793828096
Error EM:  3.967395220658631
Error EM:  2.633547277761513
Error EM:  1.7470054459372761
Error EM:  1.1584155461740622
Error EM:  0.7678809042846747
Error EM:  0.5089483332939899
Error EM:  0.33726620052432665
Error EM:  0.223453880677694
Error EM:  0.14807133995983268
Error EM:  0.09811525924642303
Error EM:  0.06499534248615328
Error EM:  0.043053376167270514
Error EM:  0.0285504716630746
Error EM:  0.018920351766172093
Error EM:  0.012522433752633285
Error EM:  0.008276023048244497
Error EM:  0.0055120861161863195
Error EM:  0.0036409838715862667
Error EM:  0.002409064225626498
Er

In [13]:
# num_cluster = 2
# M_mix, Cov_mix, p_mix = t_tools.anormal_mix(tensor_data,num_cluster,coef, var,pow_, constraint=True) 

Error EM:  241.9524287531332
Error EM:  166.35455296576694
Error EM:  209.33150313410533
Error EM:  203.98412464534962
Error EM:  169.5956987981853
Error EM:  128.0456279731306
Error EM:  91.39129896163021
Error EM:  63.12636859080435
Error EM:  42.75681769087095
Error EM:  28.61126941413917
Error EM:  18.998545632165925
Error EM:  12.55256108438231
Error EM:  8.266579162416159
Error EM:  5.432328335041886
Error EM:  3.5647618355854336
Error EM:  2.3371056800254273
Error EM:  1.531253660694042
Error EM:  1.002875803184063
Error EM:  0.656672709550288
Error EM:  0.4298856696939721
Error EM:  0.2813869582427483
Error EM:  0.18416715249798551
Error EM:  0.12053921441860757
Error EM:  0.07888818587497526
Error EM:  0.05165459233222596
Error EM:  0.03377578782675601
Error EM:  0.022133427817379626
Error EM:  0.014453257715123477
Error EM:  0.00946921518621393
Error EM:  0.006223817352563345
Error EM:  0.0040345238302322535
Error EM:  0.0026605772739787857
Error EM:  0.001725159796134716
Err

Error EM:  6.887406562018064
Error EM:  4.5811273664453
Error EM:  3.043907742577948
Error EM:  2.021051769368663
Error EM:  1.3412778661760791
Error EM:  0.8899048747992291
Error EM:  0.5902761523575312
Error EM:  0.39144622318591854
Error EM:  0.25962506364885823
Error EM:  0.17212805552765678
Error EM:  0.1141753663998153
Error EM:  0.07568752656349456
Error EM:  0.05017464020684343
Error EM:  0.03328527679517589
Error EM:  0.022074015544964385
Error EM:  0.014619457478201738
Error EM:  0.009706078061586671
Error EM:  0.006412036113780785
Error EM:  0.004280494663902344
Error EM:  0.0028201774872618503
Error EM:  0.0018906432828349412
Error EM:  0.0012318390993318322
Error EM:  0.0008070047888755669
Error EM:  0.0005643827911058046
Error EM:  0.00035231246991985197
Error EM:  0.00025880594221544686
Error EM:  0.00014024964812381566
Error EM:  9.809296322623099e-05
Error EM:  7.360224810222119e-05
Error EM:  3.2120212798957143e-05
Error EM:  6.828488047857415e-05
Error EM:  9.0356710

LinAlgError: Singular matrix

In [ ]:
print(Cov[2])

In [14]:
import pdb
# Both single and multiple cluster test against perturbation
fig1 = plt.figure()
fig2 = plt.figure()
fig3 = plt.figure()
#ax1 = fig1.add_subplot(1, 2, 1, projection='3d')
#ax2 = fig1.add_subplot(1, 2, 2, projection='3d')
ax1 = fig1.add_subplot(1, 3, 1)
ax2 = fig1.add_subplot(1, 3, 2)
ax3 = fig1.add_subplot(1, 3, 3)
ax4 = fig2.add_subplot(1, 3, 1)
ax5 = fig2.add_subplot(1, 3, 2)
ax6 = fig2.add_subplot(1, 3, 3)
ax7 = fig3.add_subplot(1, 3, 1)
ax8 = fig3.add_subplot(1, 3, 2)
ax9 = fig3.add_subplot(1, 3, 3)
#ax2.title.set_text("Estimated Trajectory")
t = np.arange(T)
for k in range(1):
    # Pick a random demo from the data
    q = int(np.random.randint(20,size=1))
    X = tensor_data[q] 
    
    p = 2 # Robot Frame
    # TP-1 trajectory from the data given initial state
    Ia = [0]
    Xa = X[0,:,:]
    #Xa = Xa #+ 0.1*np.random.randn(*Xa.shape) # perturbed trajectory
    #print(Xa)
    Xa = Xa[np.newaxis,:,:]
    
    # Given the two frames w.r.t robot frame find the trajectory
    (M_, Cov_,_,_) = t_tools.anormal_condition(M,Cov,Ia,Xa,slice_=3)
    Ib = [0,1]
    Xb = X[:,:,0:2]
    Xb[1:,:,:] = M_[:,:,0:2]

    (M_2,Cov_2,_,_) = t_tools.anormal_condition(M,Cov,Ib,Xb,slice_=1)
    
    #(p_mix_o,M_mix_o,Cov_mix_o) = t_tools.anormal_mix_condition(p_mix,M_mix,Cov_mix,Ia,Xa,slice_=3)
    #M_o = M_2#sum([p_mix_o[j]*M_mix_o[j] for j in range(len(p_mix_o))])
   
    #Xc2 = M_o[:,:,:]
    #M_2 = Xa
    
    # Trajecory tensor 3d
    Mt = M_2
    print("Mt.shape", Mt.shape)
    # Trajectory wrt robot frame
    xp = Mt[:,0,0]
    yp = Mt[:,1,0]
    zp = Mt[:,2,0]
    (xs2, ys2, zs2) = X[0,:,2] -X[0,:,1]
    (xs1, ys1, zs1) = X[0,:,2] -X[0,:,0]
    
    # Traj wrt frame 1
    xp1 = xp - xs1
    yp1 = yp - ys1
    zp1 = zp - zs1
    
    # Traj wrt frame 2
    xp2 = xp - xs2 
    yp2 = yp - ys2
    zp2 = zp - zs2 
    print("x_p.shape: ",xp.shape)
#     xline2 = Xc2[:,0,p]
#     yline2 = Xc2[:,1,p]
#     zline2 = Xc2[:,2,p]
    #ax2.plot3D(xline, yline, zline, 'red')
#     ax1.plot(t,xp,'green')
#     ax2.plot(t,yp,'blue')
#     ax3.plot(t,zp,'red')
    
    ax4.plot(t,xp1,'green')
    ax5.plot(t,yp1,'blue')
    ax6.plot(t,zp1,'red')

    ax7.plot(t,xp2,'green')
    ax8.plot(t,yp2,'blue')
    ax9.plot(t,zp2,'red')
    #ax2[0,1,2].axis('equal')
plt.show()


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Mt.shape (70, 3, 1)
x_p.shape:  (70,)


In [ ]:
.
# Testing single cluster model against trajectories taken from data
fig1 = plt.figure()
fig2 = plt.figure()
fig3 = plt.figure()
ax1 = fig1.add_subplot(1, 2, 1, projection='3d')
ax2 = fig1.add_subplot(1, 2, 2, projection='3d')
ax3 = fig2.add_subplot(1, 3, 1)
ax4 = fig2.add_subplot(1, 3, 2)
ax5 = fig2.add_subplot(1, 3, 3)
ax6 = fig3.add_subplot(1, 3, 1)
ax7 = fig3.add_subplot(1, 3, 2)
ax8 = fig3.add_subplot(1, 3, 3)
#ax2.title.set_text("Estimated Trajectory")
for k in range(20):
    # Pick a random demo from the data
    q = k #int(np.random.randint(20,size=1))
    X = tensor_data[q] 
    t = np.arange(T-1)
    p = 1
    # TP-1 trajectory from the data given initial state
    xt = X[1:,0,p]
    yt = X[1:,1,p]
    zt = X[1:,2,p] 
    ax1.plot3D(xt, yt, zt, 'green')
    #ax1.title.set_text("Given Trajectory")
    
    # Trajectory from the conditioning the array-normal
    Ia = [0]
    Xa = X[0,:,:]
    Xa = Xa[np.newaxis,:,:]
    (M_, Cov_,_,_) = t_tools.anormal_condition(M,Cov,Ia,Xa,slice_=3)
    Xc = M_[:,:,:]
    xline = Xc[:,0,p]
    yline = Xc[:,1,p]
    zline = Xc[:,2,p]
    ax3.plot(t,xt,'green')
    ax4.plot(t,yt,'blue')
    ax5.plot(t,zt,'red')
    #t = np.arange(T-)
    ax6.plot(t,xline,'green')
    ax7.plot(t,yline,'blue')
    ax8.plot(t,zline,'red')
    ax2.plot3D(xline, yline, zline, 'red')
    #ax2[0,1,2].axis('equal')
plt.show()


In [ ]:
# Testing cluster model against trajectories taken from data
fig1 = plt.figure()
fig2 = plt.figure()
fig3 = plt.figure()
ax1 = fig1.add_subplot(1, 2, 1, projection='3d')
ax2 = fig1.add_subplot(1, 2, 2, projection='3d')
ax3 = fig2.add_subplot(1, 3, 1)
ax4 = fig2.add_subplot(1, 3, 2)
ax5 = fig2.add_subplot(1, 3, 3)
ax6 = fig3.add_subplot(1, 3, 1)
ax7 = fig3.add_subplot(1, 3, 2)
ax8 = fig3.add_subplot(1, 3, 3)
#ax2.title.set_text("Estimated Trajectory")
for k in range(20):
    # Pick a random demo from the data
    q = k #int(np.random.randint(20,size=1))
    X = tensor_data[q] 
    t = np.arange(T-1)
    p = 0
    # TP-1 trajectory from the data given initial state
    xt = X[1:,0,p]
    yt = X[1:,1,p]
    zt = X[1:,2,p] 
    ax1.plot3D(xt, yt, zt, 'green')
    #ax1.title.set_text("Given Trajectory")
    
    # Trajectory from the conditioning the array-normal
    Ia = [0]
    Xa = X[0,:,:]
    Xa = Xa[np.newaxis,:,:]
    (M_, Cov_,_,_) = t_tools.anormal_condition(M,Cov,Ia,Xa,slice_=3)
    #(p_mix_o,M_mix_o,Cov_mix_o) = t_tools.anormal_mix_condition(p_mix,M_mix,Cov_mix,Ia,Xa,slice_=3)
    M_o = M_#sum([p_mix_o[j]*M_mix_o[j] for j in range(len(p_mix_o))])
    Xc = M_o[:,:,:]
    xline = Xc[:,0,p]
    yline = Xc[:,1,p]
    zline = Xc[:,2,p]
    ax3.plot(t,xt,'green')
    ax4.plot(t,yt,'blue')
    ax5.plot(t,zt,'red')
    ax6.plot(t,xline,'green')
    ax7.plot(t,yline,'blue')
    ax8.plot(t,zline,'red')
    ax2.plot3D(xline, yline, zline, 'red')
    #ax2[0,1,2].axis('equal')
plt.show()


In [ ]:
# Sampling from single cluster
fig2 = plt.figure()
fig3 = plt.figure()
ax3 = fig2.add_subplot(1, 3, 1)
ax4 = fig2.add_subplot(1, 3, 2)
ax5 = fig2.add_subplot(1, 3, 3)

t = np.arange(T)
p = 1
for k in range(20):
    # Pick a random demo from the data
    q = k #int(np.random.randint(20,size=1))
    X = tensor_data[q] 
    # TP-1 trajectory from the data given initial state
    xt = X[:,0,p]
    yt = X[:,1,p]
    zt = X[:,2,p] 
    ax3.plot(t,xt,'green')
    ax4.plot(t,yt,'blue')
    ax5.plot(t,zt,'red')

for k in range(2):    
    Xc = t_tools.anormal_sampling(M,Cov)
    #Xc=t_tools.anormal_mix_sampling(M_mix,Cov_mix, p_mix)
    xline = Xc[:,0,p]
    yline = Xc[:,1,p]
    zline = Xc[:,2,p]
    ax3.plot(t,xline,'grey')
    ax4.plot(t,yline,'grey')
    ax5.plot(t,zline,'grey')

#M_ = sum([p_mix[j]*M_mix[j] for j in range(len(p_mix))])    

xm = M_[:,0,p]
ym = M_[:,1,p]
zm = M_[:,2,p]
t = np.arange(T-1)
ax3.plot(t,xm,'black')
ax4.plot(t,ym,'black')
ax5.plot(t,zm,'black') 

plt.show()


In [ ]:
# Sampling from mixture cluster
fig2 = plt.figure()
fig3 = plt.figure()
ax3 = fig2.add_subplot(1, 3, 1)
ax4 = fig2.add_subplot(1, 3, 2)
ax5 = fig2.add_subplot(1, 3, 3)

t = np.arange(T)
p = 1
for k in range(20):
    # Pick a random demo from the data
    q = k #int(np.random.randint(20,size=1))
    X = tensor_data[q] 
    # TP-1 trajectory from the data given initial state
    xt = X[:,0,p]
    yt = X[:,1,p]
    zt = X[:,2,p] 
    ax3.plot(t,xt,'green')
    ax4.plot(t,yt,'blue')
    ax5.plot(t,zt,'red')

for k in range(5):    
    Xc = t_tools.anormal_sampling(M,Cov)
   # Xc=t_tools.anormal_mix_sampling(M_mix,Cov_mix, p_mix)
    xline = Xc[:,0,p]
    yline = Xc[:,1,p]
    zline = Xc[:,2,p]
    ax3.plot(t,xline,'grey')
    ax4.plot(t,yline,'grey')
    ax5.plot(t,zline,'grey')

M = M_#sum([p_mix[j]*M_mix[j] for j in range(len(p_mix))])    
xm = M[:,0,p]
ym = M[:,1,p]
zm = M[:,2,p]
t = np.arange(T-1)
ax3.plot(t,xm,'black')
ax4.plot(t,ym,'black')
ax5.plot(t,zm,'black') 

plt.show()


In [ ]:
# # Test sampling

# #Robot 1 trajectory from the data
# fig = plt.figure()
# ax1 = fig.add_subplot(3, 1, 1)
# ax2 = fig.add_subplot(3, 1, 2)
# ax3 = fig.add_subplot(3, 1, 3)

# t = np.arange(258)

# #plot robot-1 demo traj
# for j in range(N):
#     X = tensor_data[j]
#     xt = X[:,0,0]
#     yt = X[:,1,0]
#     zt = X[:,2,0]    
#     ax1.plot(t,xt,'green')
#     ax2.plot(t,yt,'blue')
#     ax3.plot(t,zt,'red')


# for j in range(5):
#     Xc = t_tools.anormal_sampling(M_,Cov_)
#     #Xc = t_tools.anormal_mix_sampling(M_mix,Cov_mix,p_mix)
#     xt = Xc[:,0,0]
#     yt = Xc[:,1,0]
#     zt = Xc[:,2,0]    
#     ax1.plot(t,xt,'grey')
#     ax2.plot(t,yt,'grey')
#     ax3.plot(t,zt,'grey')

# # plot mean of the robo-1 traj
# M = sum([p_mix[j]*M_mix[j] for j in range(K)])

# ax1.plot(t,M[:,0,0],'black',linewidth=1)
# ax2.plot(t,M[:,1,0],'black')
# ax3.plot(t,M[:,2,0],'black')
   
# ax3.set_ylabel("z")
# ax2.set_ylabel("y")
# ax1.set_ylabel("x")
# ax1.set_xlabel("t")
# ax1.title.set_text("Trajectory Sampling. Colored traj: Demo, Grey Traj: Sample")
# plt.show()
